In [1]:
!pip install yfinance vaderSentiment pandas

In [6]:
import pandas as pd
import yfinance as yf
from fbprophet import Prophet

# List of tickers
tickers = ["META", "TSLA", "GOOG", "VZ", "CMCSA", "T", "CHTR", "TMUS", "DIS", "NOK",
           "PLD", "AMT", "CCI", "SPG", "EQIX", "PSA", "O", "DLR", "AVB", "WELL",
           "MSFT", "AAPL", "NVDA", "AMZN", "CRM", "IBM", "ADBE", "INTC", "CSCO",
           "ORCL", "JPM", "V", "BAC", "MA", "PYPL", "BRK-B", "GS", "MS", "WFC",
           "AXP", "CVX", "XOM", "EOG", "COP", "DOW", "FCX", "KMI", "PSX", "MPC", "APD"]

# Define start and end dates
start_date = "2018-01-01"
end_date = pd.Timestamp.today().strftime("%Y-%m-%d")

# Fetch historical data
historical_data = yf.download(tickers, start=start_date, end=end_date)["Adj Close"]
# Calculate monthly returns
monthly_returns = historical_data.resample("M").last().pct_change().dropna()

# Forecast monthly returns for each stock
forecasted_returns = []
for ticker in tickers:
    if ticker in monthly_returns.columns:
        stock_returns = monthly_returns[ticker].reset_index()
        stock_returns.columns = ["ds", "y"]

        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        model.fit(stock_returns)

        future = model.make_future_dataframe(periods=12, freq="M")
        forecast = model.predict(future)

        forecasted_returns.append((ticker, forecast[["ds", "yhat"]].set_index("ds").rename(columns={"yhat": ticker})))

# Combine forecasted returns into a single DataFrame
forecasted_returns_df = pd.concat([df for _, df in forecasted_returns], axis=1)
print(forecasted_returns_df)

[*********************100%%**********************]  50 of 50 completed


                META      TSLA      GOOG        VZ     CMCSA         T  \
ds                                                                       
2019-04-30  0.070439 -0.128063 -0.018907 -0.016525  0.039969  0.002646   
2019-05-31 -0.025270  0.088663  0.027973 -0.016974 -0.019472 -0.021632   
2019-06-30  0.031193  0.310030 -0.008896  0.051969  0.013864  0.040252   
2019-07-31  0.036177  0.254229  0.120924 -0.052762  0.044272 -0.044676   
2019-08-31 -0.050944  0.035872 -0.004707  0.006466  0.029285 -0.008804   
...              ...       ...       ...       ...       ...       ...   
2025-01-31  0.037502 -0.007386  0.020973 -0.000799 -0.014566  0.052194   
2025-02-28  0.039860 -0.098540 -0.014024 -0.037515 -0.048620 -0.059153   
2025-03-31  0.057717 -0.150306  0.019435  0.011723 -0.030773 -0.017119   
2025-04-30  0.083570 -0.045398  0.066783 -0.004389 -0.007269  0.021370   
2025-05-31  0.068035 -0.180622  0.023805 -0.003088  0.016962 -0.002982   

                CHTR      TMUS       

In [8]:
import os

# Combine forecasted returns into a single DataFrame
forecasted_returns_df = pd.concat([df for _, df in forecasted_returns], axis=1)

# Define the output directory and filename
output_dir = r"C:\Users\HusnainMansoor\Documents\forecastedvalues"
csv_filename = "forecasted_monthly_returns.csv"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Construct the full file path
output_file_path = os.path.join(output_dir, csv_filename)

# Export forecasted returns to a CSV file
forecasted_returns_df.to_csv(output_file_path)

print(f"Forecasted monthly returns exported to {output_file_path}")

Forecasted monthly returns exported to C:\Users\HusnainMansoor\Documents\forecastedvalues\forecasted_monthly_returns.csv
